In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
tv = []
lmsr = []
lsmsr = []
lsmsrA = []
lsmrC = []
vig_t = []
sampled_t = []

INIT = 100

q1 = INIT*np.array([1,1]).astype(np.float64)
b = 500

q2 = INIT*np.array([1,1]).astype(np.float64)


q3 = INIT*np.array([1,1]).astype(np.float64)
prior = q3.sum()
q3_prev = np.array([.5,.5])

q4 = INIT*np.array([1,1]).astype(np.float64)
alphaG = .01

piggy_bank = 0
def get_stable_price(q,i=0):
    alpha = max(alphaG,b/q.sum())
    if False and b/q.sum() > alpha:
        q1t = q-q.max()
        return np.exp(q1t[0]/b)/np.exp(q1t/b).sum()
    else:
        #print(alpha)
        D = alpha*(np.sum(q))
        #q -= q.max()
        return alpha*np.log(np.exp(q/D).sum()) + ((q*np.exp(q[i]/D)).sum() - (q*np.exp(q/D)).sum())/(q.sum()*np.exp(q/D).sum())

for i in range(2500):
    price = .5 if i < 500 else .7
    price = price if i < 2000 else .3
    sampled = np.clip(price + .12*np.random.randn(),0,1)
    sampled_t.append(sampled)
    order_size = np.random.exponential(200)
    
    q1t = q1-q1.max()
    lmsr_p = np.exp(q1t[0]/b)/np.exp(q1t/b).sum()
    if lmsr_p < sampled:
        q1[0] += order_size
    else:
        q1[1] += order_size
        
    
    lmsr_p2 = get_stable_price(q2.copy())
    if lmsr_p2 < sampled:
        q2[0] += order_size
    else:
        q2[1] += order_size
        
    vig_t.append(get_stable_price(q2)+get_stable_price(q2,1))

    
    if False:
        lmsr_p3 = np.nan_to_num(np.clip(get_stable_price(q3.copy()),0,1))
        q3 -= q3_prev*prior
        q3_prev = np.array([lmsr_p3,1-lmsr_p3]) # (0.9*q3_prev + 0.1*
        q3 += q3_prev*prior
        lmsr_p3 = get_stable_price(q3.copy())

        if lmsr_p3 < sampled:
            q3[0] += order_size
        else:
            q3[1] += order_size
    else:
        a3 = max(alphaG,b/q3.sum())
        b3 = a3*q3.sum()
        q3t = q3-q3.max()

        lmsr_p3 = np.exp(q3t[0]/b3)/np.exp(q3t/b3).sum()
        #print(b3,np.exp(q3t[0]/b3)/np.exp(q3t/b3).sum()+np.exp(q3t[1]/b3)/np.exp(q3t/b3).sum(),vig_t[-1])

        if lmsr_p3 < sampled:
            q3[0] += order_size
        else:
            q3[1] += order_size
    a4 = max(alphaG,b/q4.sum())
    b4 = a4*q4.sum()
    q4t = q4.copy()
    qT = b4*np.log((sampled)/(1-sampled) *(np.exp(q4/b4).sum() - np.exp(q4[0]/b4))) - q4[0]

    #print(qT)
    if qT > 0 :
        order_itself = min([qT,order_size])
        q4[0] += order_itself*0.93
    else:
        order_itself = min([-qT,order_size])
        q4[1] += order_itself*0.93
    piggy_bank += order_itself*0.07
    q4t = q4-q4.max()
    lmsr_p4 = np.exp(q4t[0]/b4)/np.exp(q4t/b4).sum()
    
    tv.append(price)
    lmsr.append(lmsr_p)
    lsmsr.append(lmsr_p2)
    lsmsrA.append(lmsr_p3)
    lsmrC.append(lmsr_p4)
    q4t = q4.copy()
    q4t[0] += 1
    T1 = b4 * np.log(np.exp(q4t/b4).sum())
    q4t = q4.copy()
    q4t[1] += 1
    T2 = b4 * np.log(np.exp(q4t/b4).sum())
    T12 = T1+T2 - 2*b4 * np.log(np.exp(q4/b4).sum())

In [ ]:
get_stable_price(q2)+get_stable_price(q2,1)

In [ ]:
plt.plot(vig_t)
#plt.gca().set_yscale('log')


In [ ]:
from  scipy.ndimage import gaussian_filter1d

In [ ]:

plt.style.use('default')
#plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')

plt.figure(figsize=(12,8))

plt.plot(tv,lw=3,c='k')
plt.scatter(np.arange(len(sampled_t)),sampled_t,c='k',s=3,alpha=0.5)

plt.plot(lmsr,alpha=0.8,label='1: STD')
plt.plot(lsmsr,alpha=0.8,label='2: LS')
plt.plot(lsmsrA,alpha=0.8,label='3: STD LS-Like')

v2 = []
v2t = 0.5
for i,v in enumerate(sampled_t):
    W1 = 1/(1+np.exp(-i/1000))
    #print(W1)
    v2t = (W1) *v2t + (1-W1)*v
    v2.append(v2t)

#plt.plot(gaussian_filter1d(sampled_t,20),alpha=0.8,c='k')

plt.plot(lsmrC,label='4: STD with %')

plt.plot(v2,alpha=0.8,c='k',label='smoothing')

plt.ylim(0.1,1.0)
plt.legend(loc=1)

In [ ]:
max(q4)-min(q4),piggy_bank

In [ ]:
(max(q1)-min(q1))/sum(q1)

In [ ]:
(max(q2)-min(q2))/sum(q2)

In [ ]:
(max(q3)-min(q3))/sum(q3)

In [ ]:
(max(q4)-min(q4))/sum(q4)

In [ ]:
(b/q3.sum())*q3.sum()

In [ ]:
sampled

In [ ]:
np.random.randn()